In [95]:
import random
import numpy as np
# Function to generate random unfriendly relationship matrix
def generate_unfriendly_matrix(num_people, num_unfriendly, groups): # TODO
    matrix = [[0 for _ in range(num_people)] for _ in range(num_people)]
    while num_unfriendly > 0:
        i = random.randint(0, num_people - 1)
        j = random.randint(0, num_people - 1)
        # Ensure i != j and avoid duplicate unfriendly relationships
        if i != j and matrix[i][j] == 0:
            flag = False
            for group in groups:
                if i in group and j in group:
                    flag = True
                    break
            if not flag:
                matrix[i][j] = 1
                matrix[j][i] = 1 # Make it symmetric
                num_unfriendly -= 1
    # print("training data finished")
    return matrix

def create_groups(num_people, group_size):
    groups = []
    num_groups = num_people // group_size
    # Loop through the number of groups
    for _ in range(num_groups):
        # Create a new group list
        group = []

        # Fill the group with people (up to group_size)
        for _ in range(group_size):
            if num_people > 0:  # from 1 to num_people
                group.append(num_people)  # Add person names (example)
                num_people -= 1

        # Add the filled group to the groups list
        groups.append(group)

    return groups

import torch

#gpt list view
# def calculate_conflict_cost(seating_arrangement, unfriendly_matrix, groups):
#     alpha = 0.5
#     loss = 0

#     ROWS, COLS = 20, 20

#     # Convert seating_arrangement to a PyTorch tensor if it is a list
#     if isinstance(seating_arrangement, list):
#         seating_arrangement = torch.tensor(seating_arrangement)

#     # Reshape the seating_arrangement to the desired shape
#     seating_arrangement = seating_arrangement.view(ROWS, COLS)

#     # Initialize the loss tensor
#     loss = torch.tensor(0.0, dtype=torch.float, requires_grad=True)

#     for r in range(ROWS):
#         for c in range(COLS):
#             person = seating_arrangement[r, c].item()
#             if r > 0:  # Check the person above
#                 if unfriendly_matrix[person, seating_arrangement[r - 1, c].item()] == 1:
#                     loss = loss + 1
#             if r < ROWS - 1:  # Check the person below
#                 if unfriendly_matrix[person, seating_arrangement[r + 1, c].item()] == 1:
#                     loss = loss + 1
#             if c > 0:  # Check the person to the left
#                 if unfriendly_matrix[person, seating_arrangement[r, c - 1].item()] == 1:
#                     loss = loss + 1
#             if c < COLS - 1:  # Check the person to the right
#                 if unfriendly_matrix[person, seating_arrangement[r, c + 1].item()] == 1:
#                     loss = loss + 1

#     return loss

#my main
# def calculate_conflict_cost(seating_arrangement, unfriendly_matrix, groups):
#     alpha=0.5
#     #conflict_cost=0
#     loss=0
#     #conflicts

#     ROWS,COLS=4,6
# # def loss_function(arrangement, relationship_matrix):
#     seating_arrangement = seating_arrangement.view(ROWS, COLS)
# #     #print(arrangement.view(ROWS, COLS))
#     loss = torch.tensor(0.0, dtype=torch.float, requires_grad=True)
#     for r in range(ROWS):
#         for c in range(COLS):
#             person = seating_arrangement[r, c].item()
#             if r > 0:  # Check the person above
#                 if unfriendly_matrix[person, seating_arrangement[r - 1, c].item()] == 1:
#                     loss = loss + 1
#             if r < ROWS - 1:  # Check the person below
#                 if unfriendly_matrix[person, seating_arrangement[r + 1, c].item()] == 1:
#                     loss = loss + 1
#             if c > 0:  # Check the person to the left
#                 if unfriendly_matrix[person, seating_arrangement[r, c - 1].item()] == 1:
#                     loss = loss + 1
#             if c < COLS - 1:  # Check the person to the right
#                 if unfriendly_matrix[person, seating_arrangement[r, c + 1].item()] == 1:
#                     loss = loss + 1

    
#     return loss
#     # groups

    # neighbering_cost = 0
    # for i in range(len(groups)):
    #     group = groups[i]
    #     # neighbering_cost = 0
    #     dictance = 0
    #     for j in range(len(group)):
    #         for k in range(j+1, len(group)):
    #             j_inx = seating_arrangement.index(j)
    #             k_inx = seating_arrangement.index(k)
    #             distance += abs(k_inx - j_inx)
    #     if distance > 20: # sum (index difirences)
    #         neighbering_cost += (distance - 20)


    # total_cost = alpha * conflict_cost + (1 - alpha) * neighbering_cost
    # return total_cost


#main
def calculate_conflict_cost(seating_arrangement, unfriendly_matrix, groups):
    alpha = 0.5
    conflict_cost = 0
    # conflicts
    for i in range(len(seating_arrangement)):
        for j in range(i + 1, len(seating_arrangement)): # Avoid double counting
            distance = abs(i - j)
            weight = 1.0 / (distance**2 + 1) # Avoid division by zero and give higher weight to closer positions
            cost = unfriendly_matrix[seating_arrangement[i]][seating_arrangement[j]] * weight
            conflict_cost += cost
    # groups


    neighboring_cost = 0
    # Convert seating_arrangement to a list if it's not already
    if isinstance(seating_arrangement, np.ndarray):
        seating_arrangement = seating_arrangement.tolist()
    elif isinstance(seating_arrangement, torch.Tensor):
        seating_arrangement = seating_arrangement.numpy().tolist()

    # Flatten the seating arrangement if it's not already flat
    if isinstance(seating_arrangement[0], list):
        seating_arrangement = [item for sublist in seating_arrangement for item in sublist]

    for group in groups:
        distance = 0
        for j in range(len(group)):
            for k in range(j + 1, len(group)):
                person_j = group[j]
                person_k = group[k]
                if person_j in seating_arrangement and person_k in seating_arrangement:
                    j_inx = seating_arrangement.index(person_j)
                    k_inx = seating_arrangement.index(person_k)
                    distance += abs(k_inx - j_inx)
        if distance > 20:  # sum (index differences)
            neighboring_cost += (distance - 20)
    


    # neighbering_cost = 0
    # for i in range(len(groups)):
    #     group = groups[i]
    #     # neighbering_cost = 0
    #     dictance = 0
    #     for j in range(len(group)):
    #         for k in range(j + 1, len(group)):
    #             j_inx = seating_arrangement.index(j)
    #             k_inx = seating_arrangement.index(k)
    #             distance += abs(k_inx - j_inx)
    #     if distance > 20: # sum (index difirences)
    #         neighbering_cost += (distance - 20)


    total_cost = alpha * conflict_cost/600 + (1 - alpha) * neighboring_cost/1996
    return total_cost


In [96]:
# def calculate_neighbering_cost(seating_arrangement, groups):
#     neighboring_cost = 0
 

#     # Convert seating_arrangement to a list if it's not already
#     if isinstance(seating_arrangement, np.ndarray):
#         seating_arrangement = seating_arrangement.tolist()
#     elif isinstance(seating_arrangement, torch.Tensor):
#         seating_arrangement = seating_arrangement.numpy().tolist()

#     # Flatten the seating arrangement if it's not already flat
#     if isinstance(seating_arrangement[0], list):
#         seating_arrangement = [item for sublist in seating_arrangement for item in sublist]

#     for group in groups:
#         distance = 0
#         for j in range(len(group)):
#             for k in range(j + 1, len(group)):
#                 person_j = group[j]
#                 person_k = group[k]
#                 if person_j in seating_arrangement and person_k in seating_arrangement:
#                     j_inx = seating_arrangement.index(person_j)
#                     k_inx = seating_arrangement.index(person_k)
#                     distance += abs(k_inx - j_inx)
#         if distance > 20:  # sum (index differences)
#             neighboring_cost += (distance - 20)
    
#     return neighboring_cost

# # # Example usage
# # predicted_seating_arrangement = [i for i in range(24)]
# # groups = [[0, 1, 2], [3, 4, 5]]  # Example groups
# # unfriendly_matrix = np.zeros((24, 24))  # Example matrix

# # neighboring_cost = calculate_neighbering_cost(predicted_seating_arrangement, groups)
# # print("neighboring_cost", neighboring_cost)


In [97]:

# #gpts second reshape
# import numpy as np
# import torch

# def calculate_conflict_cost(seating_arrangement, unfriendly_matrix, groups):
#     alpha = 0.5
#     loss = 0

#     ROWS, COLS = 20, 20

#     # Convert seating_arrangement to a NumPy array if it is a list
#     if isinstance(seating_arrangement, list):
#         seating_arrangement = np.array(seating_arrangement)

#     # Reshape the seating_arrangement to the desired shape
#     seating_arrangement = seating_arrangement.reshape(ROWS, COLS)

#     # Initialize the loss tensor
#     loss = torch.tensor(0.0, dtype=torch.float, requires_grad=True)

#     for r in range(ROWS):
#         for c in range(COLS):
#             person = seating_arrangement[r, c]
#             if r > 0:  # Check the person above
#                 if unfriendly_matrix[person, seating_arrangement[r - 1, c]] == 1:
#                     loss = loss + 1
#             if r < ROWS - 1:  # Check the person below
#                 if unfriendly_matrix[person, seating_arrangement[r + 1, c]] == 1:
#                     loss = loss + 1
#             if c > 0:  # Check the person to the left
#                 if unfriendly_matrix[person, seating_arrangement[r, c - 1]] == 1:
#                     loss = loss + 1
#             if c < COLS - 1:  # Check the person to the right
#                 if unfriendly_matrix[person, seating_arrangement[r, c + 1]] == 1:
#                     loss = loss + 1

#     return loss


In [98]:
# import numpy as np
# import torch

# def calculate_conflict_cost(seating_arrangement, unfriendly_matrix, groups):
#     alpha = 0.5
#     loss = 0

#     ROWS, COLS = 20, 20

#     # Convert seating_arrangement and unfriendly_matrix to NumPy arrays if they are lists
#     if isinstance(seating_arrangement, list):
#         seating_arrangement = np.array(seating_arrangement)
#     if isinstance(unfriendly_matrix, list):
#         unfriendly_matrix = np.array(unfriendly_matrix)

#     # Reshape the seating_arrangement to the desired shape
#     seating_arrangement = seating_arrangement.reshape(ROWS, COLS)

#     # Initialize the loss tensor
#     loss = torch.tensor(0.0, dtype=torch.float, requires_grad=True)

#     for r in range(ROWS):
#         for c in range(COLS):
#             person = seating_arrangement[r, c]
#             if r > 0:  # Check the person above
#                 if unfriendly_matrix[person, seating_arrangement[r - 1, c]] == 1:
#                     loss = loss + 1
#             if r < ROWS - 1:  # Check the person below
#                 if unfriendly_matrix[person, seating_arrangement[r + 1, c]] == 1:
#                     loss = loss + 1
#             if c > 0:  # Check the person to the left
#                 if unfriendly_matrix[person, seating_arrangement[r, c - 1]] == 1:
#                     loss = loss + 1
#             if c < COLS - 1:  # Check the person to the right
#                 if unfriendly_matrix[person, seating_arrangement[r, c + 1]] == 1:
#                     loss = loss + 1

#     #return loss

#     # groups

#     neighbering_cost = 0
#     for i in range(len(groups)):
#         group = groups[i]
#         # neighbering_cost = 0
#         dictance = 0
#         for j in range(len(group)):
#             for k in range(j+1, len(group)):
#                 j_inx = seating_arrangement.index(j)
#                 k_inx = seating_arrangement.index(k)
#                 distance += abs(k_inx - j_inx)
#         if distance > 20: # sum (index difirences)
#             neighbering_cost += (distance - 20)


#     total_cost = alpha * conflict_cost + (1 - alpha) * neighbering_cost
#     return total_cost


# # Example usage
# predicted_seating_arrangement = [i for i in range(400)]
# unfriendly_matrix = [[0, 1, 0, 1], [1, 0, 1, 0], [0, 1, 0, 1], [1, 0, 1, 0]]  # Example matrix
# groups = []  # Example groups

# # Adjust to match the test data dimensions
# unfriendly_matrix = np.zeros((400, 400))
# unfriendly_matrix[0, 1] = 1
# unfriendly_matrix[1, 0] = 1
# unfriendly_matrix[2, 3] = 1
# unfriendly_matrix[3, 2] = 1

# conflict = calculate_conflict_cost(predicted_seating_arrangement, unfriendly_matrix, groups)
# print("conflict", conflict)


In [99]:

# generate training data
def generate_training_data(num_people, num_unfriendly, num_samples, groups):
    training_data = []
    for _ in range(num_samples):
        _unfriendly_matrix = generate_unfriendly_matrix(num_people, num_unfriendly, groups)
        training_data.append(_unfriendly_matrix)
    return training_data

def convert_to_permutation(output):
    _, indices = torch.sort(output, descending=True)
    return indices + 1  # تبدیل به دامنه 1 تا 24
def convert_index_to_names(predicted_seating_arrangement):
    first_names = ["Ali","mina","hoda","tina", "sasan", "mahdi", "Zahra", "Reza", "Sara","saman","sadegh", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya"]
    last_names = ["Ahmadi", "Hosseini", "Karimi","mahdavi", "hamedi", "ashian","shayan","sadeghi","sam", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam"]
    random.seed(600)
    random_names = set()
    while len(random_names) < 400:
        first_name = random.choice(first_names)
        last_name = random.choice(last_names)
        random_names.add(f"{first_name} {last_name}")

    random_names = list(random_names) # primary list of names
    seating_arrangement = [[0 for _ in range(20)] for _ in range(20)]
    t = 0
    for i in range(20):
        for j in range(20):
            seating_arrangement[i][j] = random_names[predicted_seating_arrangement[t]]
            t += 1
    return seating_arrangement

In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
# Neural network architecture
class SeatingArrangementNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SeatingArrangementNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.flat = nn.Flatten(0, -1)
    def forward(self, x):
        x = self.flat(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Hyperparameters
num_people = 400
num_unfriendly_pairs = 600
learning_rate = 0.001
num_epochs = 3
num_samples = 100
group_size = 5
# Model definition
input_size = num_people * num_people
hidden_size = 24
output_size = num_people # Output represents predicted seating order
model = SeatingArrangementNet(input_size, hidden_size, output_size)
# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = calculate_conflict_cost
# Load training data
num_groups = num_people // group_size
groups = create_groups(num_people, group_size)
print(groups)
urm_data = generate_training_data(num_people, num_unfriendly_pairs, num_samples, groups)
urm_data = torch.tensor(urm_data).float()
print(len(urm_data))
# Training loop
for epoch in range(num_epochs):
    for i in range(0, len(urm_data)):

        urm_tensor = torch.tensor(urm_data[i])
        predictions = model(urm_tensor.type(torch.FloatTensor))
        predictions = predictions.argsort(dim=0)
        permuted_outputs = torch.stack([convert_to_permutation(output) for output in predictions])
        permuted_outputs = [element - 1 for element in permuted_outputs]
        loss = loss_fn(permuted_outputs, urm_tensor, groups) # Update loss function arguments
        optimizer.zero_grad()
        loss.requires_grad = True
        loss.backward()
        optimizer.step()
        # print("urm[i] is done")

    print(f"Epoch: {epoch+1}/{num_epochs}")
print("Training complete!", )

# test
unfreindly_matrix = generate_unfriendly_matrix(num_people, num_unfriendly_pairs, groups)
test_urm_tensor = torch.tensor(unfreindly_matrix)
predicted_seating_arrangement = model(test_urm_tensor.flatten().float())

predicted_seating_arrangement = convert_to_permutation(predicted_seating_arrangement)
predicted_seating_arrangement = [element - 1 for element in predicted_seating_arrangement]
# names = convert_index_to_names(predicted_seating_arrangement)
conflict = loss_fn(predicted_seating_arrangement, unfreindly_matrix, groups)
# print("names: ", names, "conflict:", conflict)
print("conflict", conflict)

[[400, 399, 398, 397, 396], [395, 394, 393, 392, 391], [390, 389, 388, 387, 386], [385, 384, 383, 382, 381], [380, 379, 378, 377, 376], [375, 374, 373, 372, 371], [370, 369, 368, 367, 366], [365, 364, 363, 362, 361], [360, 359, 358, 357, 356], [355, 354, 353, 352, 351], [350, 349, 348, 347, 346], [345, 344, 343, 342, 341], [340, 339, 338, 337, 336], [335, 334, 333, 332, 331], [330, 329, 328, 327, 326], [325, 324, 323, 322, 321], [320, 319, 318, 317, 316], [315, 314, 313, 312, 311], [310, 309, 308, 307, 306], [305, 304, 303, 302, 301], [300, 299, 298, 297, 296], [295, 294, 293, 292, 291], [290, 289, 288, 287, 286], [285, 284, 283, 282, 281], [280, 279, 278, 277, 276], [275, 274, 273, 272, 271], [270, 269, 268, 267, 266], [265, 264, 263, 262, 261], [260, 259, 258, 257, 256], [255, 254, 253, 252, 251], [250, 249, 248, 247, 246], [245, 244, 243, 242, 241], [240, 239, 238, 237, 236], [235, 234, 233, 232, 231], [230, 229, 228, 227, 226], [225, 224, 223, 222, 221], [220, 219, 218, 217, 216], 

C:\Users\saeed\AppData\Local\Temp\ipykernel_5132\3792754014.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  urm_tensor = torch.tensor(urm_data[i])


Epoch: 1/3
Epoch: 2/3
Epoch: 3/3
Training complete!
conflict 26.0802752085158


In [102]:
names = convert_index_to_names(predicted_seating_arrangement)
print("names: ", names, "conflict:", conflict)